In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="7" 

device = 'cuda'

## Load model

In [2]:
from torchvision.models import resnet18, resnet50,  mobilenet_v2

model = mobilenet_v2(pretrained = True)

In [3]:
model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

## Collect model statistics

In [4]:
from flopco import FlopCo

model.to(device)
model_stats = FlopCo(model, img_size = (1, 3, 224, 224), device = device)

## Create compressor

In [5]:
from musco.pytorch import CompressorVBMF, CompressorPR, CompressorManual
import torch.nn as nn

In [21]:
import torch
import torch.nn.functional as F

def calibrate(model, device, train_loader, max_iters = 1000,
              freeze_lnames = None):

    model.to(device).train()
    for pname, p in model.named_parameters():
        
        if pname.strip('.weight').strip('.bias')  in freeze_lnames:
            p.requires_grad = False

    with torch.no_grad():
        for i, (data, _) in enumerate(loaders['train']):
            _ = model(data.to(device))

            if i%50 == 0:
                print('hey')

            if i > max_iters:
                break

            del data
            torch.cuda.empty_cache()
            
    model.eval()
    return model


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i,(data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            
            output = model(data)
            
            
            test_loss += F.nll_loss(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            if i % 50 == 0:
                print('hop')
            
    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    
    del data
    torch.cuda.empty_cache()
    
    return  accuracy


import sys
sys.path.append('/workspace/home/jgusak/musco/')

import dataloaders

loaders = dataloaders.get_loader(batch_size=128, data_name='imagenet',
                       data_root="/workspace/raid/data/datasets/",
                       train = True, val=True, num_workers = 8)

Building imagenet data loader with 8 workers


In [22]:
lnames_to_compress = [k for k in model_stats.flops.keys() if\
                      model_stats.ltypes[k]['type'] == nn.Conv2d and\
                      model_stats.ltypes[k]['kernel_size'] == (1, 1)
                     ]

In [23]:
from musco.pytorch.compressor.rank_selection.estimator import estimate_rank_for_compression_rate, estimate_vbmf_ranks
from collections import defaultdict

wf = 0.55
nx = 1.

max_ranks = defaultdict()

lnames_compress_me = []
for mname, m in model.named_modules():
    if mname in lnames_to_compress:
        lname = mname
        _, cin, _, _ = model_stats.input_shapes[lname][0]
        _, cout, _, _ = model_stats.output_shapes[lname][0]
        kernel_size = model_stats.ltypes[lname]['kernel_size']

        tensor_shape = (cout, cin, *kernel_size)
        r_pr = estimate_rank_for_compression_rate(tensor_shape[:2], rate = nx, key = 'svd')
        
        r_vbmf = estimate_vbmf_ranks(m.weight.data[:, :, 0, 0], k = wf)
        
        max_ranks[lname] = int(r_pr)

        print('\n', lname, tensor_shape, r_pr, r_vbmf)
        if r_pr > r_vbmf:
            lnames_compress_me.append(lname)
            print('===== COMPRESS ME ===== {} times\n'.format(r_pr/r_vbmf))
        else:
            print('===== DO NOT COMPRESS ME =====\n')

-------------------- SVD time:  0.0002949237823486328
------------------- VBMF time :  0.0019257068634033203

 features.1.conv.1 (16, 32, 1, 1) 10.0 16
===== DO NOT COMPRESS ME =====

-------------------- SVD time:  0.0007646083831787109
-------------------- SVD time:  0.00040411949157714844
------------------- VBMF time :  0.0019807815551757812

 features.2.conv.0.0 (96, 16, 1, 1) 13.0 16
===== DO NOT COMPRESS ME =====

-------------------- SVD time:  0.0007398128509521484
------------------- VBMF time :  0.002353668212890625
EVBMF returned 0 rank

 features.2.conv.2 (24, 96, 1, 1) 19.0 24
===== DO NOT COMPRESS ME =====

-------------------- SVD time:  0.0024864673614501953
-------------------- SVD time:  0.0017256736755371094
------------------- VBMF time :  0.0027618408203125

 features.3.conv.0.0 (144, 24, 1, 1) 20.0 11
===== COMPRESS ME ===== 1.8181818181818181 times

-------------------- SVD time:  0.0019388198852539062
------------------- VBMF time :  0.00302886962890625

 featu

In [24]:
max_ranks

defaultdict(None,
            {'features.1.conv.1': 10,
             'features.2.conv.0.0': 13,
             'features.2.conv.2': 19,
             'features.3.conv.0.0': 20,
             'features.3.conv.2': 20,
             'features.4.conv.0.0': 20,
             'features.4.conv.2': 26,
             'features.5.conv.0.0': 27,
             'features.5.conv.2': 27,
             'features.6.conv.0.0': 27,
             'features.6.conv.2': 27,
             'features.7.conv.0.0': 27,
             'features.7.conv.2': 48,
             'features.8.conv.0.0': 54,
             'features.8.conv.2': 54,
             'features.9.conv.0.0': 54,
             'features.9.conv.2': 54,
             'features.10.conv.0.0': 54,
             'features.10.conv.2': 54,
             'features.11.conv.0.0': 54,
             'features.11.conv.2': 76,
             'features.12.conv.0.0': 82,
             'features.12.conv.2': 82,
             'features.13.conv.0.0': 82,
             'features.13.conv.2': 82,


In [26]:
import copy 

top1_init = 71.88
eps = 0.01


min_ranks = {k:2 for k in max_ranks.keys()}

curr_ranks = copy.deepcopy(max_ranks)


for idx,lname in enumerate(lnames_to_compress):
    print(lname)
    
    curr = max_ranks[lname]
    curr_max = max_ranks[lname]
    curr_min = min_ranks[lname]
    
    for i in range(4):
        print(curr_min, curr, curr_max)
        
        ranks =  {k:None for k in model_stats.flops.keys() if k not in [lname]}
        ranks[lname] = curr
    
        compressor = CompressorManual(model, model_stats,\
                              ranks = ranks,
                              ft_every = 1,\
                              nglobal_compress_iters = 1)

        print("\n Compress")
        compressor.compression_step()

        print("\n Calibrate")
        compressor.model = calibrate(compressor.compressed_model,\
                                     device, loaders['train'],\
                                     freeze_lnames = lnames_to_compress[:idx])

        print("\n Test")
        top1 = test(compressor.compressed_model, device, loaders['val'])

        if top1 + eps < top1_init:
            if i == 0:
                print('Bad layer to compress')
                break
            else:
                curr_min = curr
                curr = curr + (curr_max - curr)//2
        else:
            curr_max = curr
            curr = curr - (curr - curr_min)//2
            
        
    print('\n Fine-tune')

features.1.conv.1
2 10 10

 Compress
features.1.conv.1 svd 10 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1162, Accuracy: 33654/50000 (67.31%)

Bad layer to compress

 Fine-tune
features.2.conv.0.0
2 13 13

 Compress
features.2.conv.0.0 svd 13 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1270, Accuracy: 35843/50000 (71.69%)

Bad layer to compress

 Fine-tune
features.2.conv.2
2 19 19

 Compress
features.2.conv.2 svd 19 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1222, Accuracy: 34893/50000 (6

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1281, Accuracy: 35881/50000 (71.76%)

Bad layer to compress

 Fine-tune
features.6.conv.0.0
2 27 27

 Compress
features.6.conv.0.0 svd 27 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1276, Accuracy: 35870/50000 (71.74%)

Bad layer to compress

 Fine-tune
features.6.conv.2
2 27 27

 Compress
features.6.conv.2 svd 27 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1279, Accuracy: 35861/50000 (71.72%)

Bad layer to compress

 Fine-tune
features.7.conv.0.0
2 27 27

 Compress
features.7.conv.0.0 svd 27 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1233, Accuracy: 34969/50000 (69.94%)

Bad layer to compress

 Fine-tune
features.7.conv.2
2 48 48

 Compress
features.7.conv.2 svd 48 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
he

hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1260, Accuracy: 35652/50000 (71.30%)

Bad layer to compress

 Fine-tune
features.16.conv.0.0
2 137 137

 Compress
features.16.conv.0.0 svd 137 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1267, Accuracy: 35745/50000 (71.49%)

Bad layer to compress

 Fine-tune
features.16.conv.2
2 137 137

 Compress
features.16.conv.2 svd 137 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1267, Accuracy: 35626/50000 (71.25%)

Bad layer to compress

 Fine-tune
features.17.conv.0.0
2 137 137

 Compress
features.17.conv.0.0 svd 137 

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1234, Accuracy: 35430/50000 (70.86%)

Bad layer to compress

 Fine-tune
features.17.conv.2
2 240 240

 Compress
features.17.conv.2 svd 240 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1235, Accuracy: 35496/50000 (70.99%)

Bad layer to compress

 Fine-tune
features.18.0
2 256 256

 Compress
features.18.0 svd 256 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1278, Accuracy: 35849/50000 (71.70%)

Bad layer to compress

 Fine-tune


In [10]:
for lname in lnames_to_compress:
    ranks =  {k:None for k in model_stats.flops.keys() if k not in [lname]}
    ranks[lname] = max_ranks[lname]
    
    compressor = CompressorManual(model, model_stats,\
                          ranks = ranks,
                          ft_every = 1,\
                          nglobal_compress_iters = 1)

    while not compressor.done:
        print("\n Compress")
        compressor.compression_step()
        
        print("\n Calibrate")
        compressor.model = calibrate(compressor.compressed_model, device, loaders['train'])
        
        print("\n Test")
        test(compressor.compressed_model, device, loaders['val'])

        print('\n Fine-tune')


 Compress
features.1.conv.1 svd 10 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1164, Accuracy: 33676/50000 (67.35%)


 Fine-tune

 Compress
features.2.conv.0.0 svd 13 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1269, Accuracy: 35826/50000 (71.65%)


 Fine-tune

 Compress
features.2.conv.2 svd 19 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1222, Accuracy: 34932/50000 (69.86%)


 Fine-tune

 Compress
features.3.conv.0.0 svd 20 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
h

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1277, Accuracy: 35848/50000 (71.70%)


 Fine-tune

 Compress
features.3.conv.2 svd 20 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey


/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1272, Accuracy: 35690/50000 (71.38%)


 Fine-tune

 Compress
features.4.conv.0.0 svd 20 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1260, Accuracy: 35556/50000 (71.11%)


 Fine-tune

 Compress
features.4.conv.2 svd 26 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1227, Accuracy: 34865/50000 (69.73%)


 Fine-tune

 Compress
features.5.conv.0.0 svd 27 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1278, Accuracy: 35917/50000 (71.83%)


 Fine-tune

 Compress
features.8.conv.2 svd 54 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1277, Accuracy: 35874/50000 (71.75%)


 Fine-tune

 Compress
features.9.conv.0.0 svd 54 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set: Average loss: -0.1279, Accuracy: 35940/50000 (71.88%)


 Fine-tune

 Compress
features.9.conv.2 svd 54 <class 'torch.nn.modules.conv.Conv2d'> None None

 Calibrate
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey

 Test
hop
hop
hop
hop
hop
hop
hop
hop

Test set

KeyboardInterrupt: 

In [ ]:
compressor.compressed_model

In [134]:
stats_compressed = FlopCo(compressor.compressed_model.to(device), device = device)

In [135]:
1/(model_stats.total_flops / stats_compressed.total_flops)

0.7103806317463074